In [ ]:
######## Image Object Detection Using Tensorflow-trained Classifier #########
#
#
import os
import cv2
import numpy as np
import tensorflow as tf
import sys

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

# Import utilites
from utils import label_map_util
from utils import visualization_utils as vis_util

# Name of the directory containing the object detection module we're using
MODEL_NAME = 'inference_graph'
IMAGE_NAME = 'test1.jpg'

# Grab path to current working directory
CWD_PATH = os.getcwd()

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,'training','labelmap.pbtxt')

# Path to image
PATH_TO_IMAGE = os.path.join(CWD_PATH,IMAGE_NAME)

# Number of classes the object detector can identify
NUM_CLASSES = 3

# Load the label map.
# Label maps map indices to category names, so that when our convolution
# network predicts `5`, we know that this corresponds to `king`.
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

# Load image using OpenCV and
# expand image dimensions to have shape: [1, None, None, 3]
# i.e. a single-column array, where each item in the column has the pixel RGB value
image = cv2.imread(PATH_TO_IMAGE)
image_expanded = np.expand_dims(image, axis=0)

# Perform the actual detection by running the model with the image as input
(boxes, scores, classes, num) = sess.run(
    [detection_boxes, detection_scores, detection_classes, num_detections],
    feed_dict={image_tensor: image_expanded})

# Draw the results of the detection (aka 'visulaize the results')

vis_util.visualize_boxes_and_labels_on_image_array(
    image,
    np.squeeze(boxes),
    np.squeeze(classes).astype(np.int32),
    np.squeeze(scores),
    category_index,
    use_normalized_coordinates=True,
    line_thickness=8,
    min_score_thresh=0.80)

# All the results have been drawn on image. Now display the image.
cv2.imshow('Object detector', image)
cv2.imwrite('image.png',image)

print(boxes, scores, classes, num)



# Press any key to close the image
cv2.waitKey(0)

# Clean up
cv2.destroyAllWindows()


[[[4.2449531e-01 4.3127677e-01 5.7509965e-01 6.0267133e-01]
  [4.2804861e-01 6.1106104e-01 5.9759027e-01 7.6233977e-01]
  [4.2791298e-01 7.7091557e-01 6.0028702e-01 9.1419631e-01]
  ...
  [4.7449341e-01 6.2478465e-01 5.6021851e-01 7.4236226e-01]
  [4.4140425e-01 4.7024092e-04 6.2381166e-01 7.6517604e-02]
  [8.0926394e-01 8.3142817e-01 9.8594534e-01 1.0000000e+00]]] [[0.32750386 0.25794616 0.23758654 0.22973001 0.14786977 0.10687122
  0.08888736 0.0620078  0.05856376 0.05801065 0.05482724 0.05049904
  0.04754966 0.04736524 0.03870972 0.03807076 0.03802888 0.03656379
  0.03377446 0.03161149 0.03125449 0.03109143 0.03054385 0.02994164
  0.02788263 0.02700347 0.02694926 0.02604588 0.02548825 0.02500417
  0.02472288 0.02430594 0.02423205 0.02364527 0.02356401 0.02323175
  0.02317419 0.02227239 0.02131268 0.02014795 0.02013183 0.01988929
  0.01966224 0.01926176 0.0188837  0.01863389 0.0182922  0.0180329
  0.0173917  0.01728228 0.01728166 0.01727124 0.01643231 0.01631282
  0.01615007 0.015586